In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
extracted_coefficients_directory = '../../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
z = np.load(z_path)
tBB = B_zeta.T.dot(B_zeta)
n = B_zeta.shape[0]
p = B_zeta.shape[1]

In [ ]:
X = B_zeta

In [ ]:
hmc_code = '''
functions {
    vector S_xtheta(vector lambda, matrix X, int p, int n) {
        vector[n] s_is;
        for (N in 1:n) {
          s_is[N] = 1/sqrt(1 + sum((row(X, N).*square(to_row_vector(lambda))).*(row(X, N)))); 
        } 
        return(s_is);
    }
    }
data {
  int<lower=1> n; // Number of data
  int<lower=1> p; // Number of covariates
  matrix[n,p] X;  // n-by-p design matrix
  real y[n];      // n-dimensional response vector
}


parameters {
  vector[p] beta;
  vector<lower=0>[p] lambda;
  real<lower=0> tau;
}

transformed parameters {
  vector[n] theta ;
  vector[n] S ;
  vector[n] Var ;
  S = S_xtheta(lambda, X, p, n);
  theta = S .* (X * beta);
  Var = square(S);
}

model {
  tau ~ cauchy(0, 1);
  lambda ~ cauchy(0, tau);
  beta ~ normal(0, square(lambda)); 
  y ~ normal(theta, Var);
}'''

In [ ]:
hmc_dat = {'n': n,
           'p': p,
           'X': X,
           'y': z}

In [ ]:
sm = pystan.StanModel(model_code = hmc_code, verbose = True)
print('finished compiling')
print('start sampling')
fit = sm.sampling(data = hmc_dat,
                  iter = 5000,
                  chains = 1, 
                  warmup = 300,
                  verbose = True)
print('finished sampling')

In [ ]:
pystan.check_hmc_diagnostics(fit)

In [ ]:
samples = fit.extract(permuted=True)

In [ ]:
plt.plot(samples['beta'])

In [ ]:
samples['tau'].shape

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('1000_samples_filtered.p', 'wb') as fp:
    pickle.dump(samples, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
la = fit.extract(permuted=True) 
a = fit.extract(permuted=False)

In [ ]:
fit.plot()

In [ ]:
Lambda = samples['lambda']
beta = samples['beta']
tau = samples['tau']

In [ ]:
np.savetxt('Lambda_pystan.csv', Lambda, delimiter =',')
np.savetxt('beta_pystan.csv', beta, delimiter =',')
np.savetxt('tau_pystan.csv', tau, delimiter =',')

In [ ]:
tau